<p align="center"><b>DEEP LEARNING PARA PROCESSAMENTO DE LINGUAGEM E VISÃO</b></p>
<p align="justify"><b>Turma 1B - 2022/1</b></p>
<p><b>Trilha 6:</b> Tarefa (Peso 3)</p>
<p></p>
<p align='justify'>A mineração de textos é uma das principais atividades do processamento de linguagem natural. Diferentes domínios podem construir aplicações para entender melhor os dados textuais, como grandes portais jornalísticos, que constroem sistemas inteligentes para categorizar notícias.</p>
<p align='center'><img src=https://dhg1h5j42swfq.cloudfront.net/2021/05/11111019/image-255.png></p>
<p align='justify'>Nesta tarefa, você deverá desenvolver um categorizador de notícias utilizando redes neurais recorrentes. Para isso, utilize o conjunto de dados <i>Reuters</i> presente nativamente no TensorFlow-Keras.</p>

